In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [2]:
features = np.asarray(np.load("/home/sorozco0612/dev/alan_bot/data_processing/road_data/features.npy"))
labels = np.asarray(np.load("/home/sorozco0612/dev/alan_bot/data_processing/road_data/labels.npy"))
#features = features.reshape(list(features.shape)+ [1])

print(features.shape)
print(labels.shape)


x_train, x_test, y_train, y_test = train_test_split(features,labels)


(8975, 432, 614, 3)
(8975, 2)


In [3]:
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu', input_shape=(432,614,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(2))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 430, 612, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 215, 306, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 213, 304, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 106, 152, 64)      0         
_________________________________________________________________
flatten (Flatten)            (None, 1031168)           0         
_________________________________________________________________
dense (Dense)                (None, 64)                65994816  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1

In [4]:
model.compile(optimizer='adam',
              loss = tf.keras.losses.MeanSquaredError(),
              metrics=['mean_squared_error'])

In [5]:
fit = model.fit(x_train,y_train,epochs=120,validation_data=(x_test,y_test))

Epoch 1/120
211/211 [==============================] - 46s 193ms/step - loss: 2055480.7347 - mean_squared_error: 2055480.7347 - val_loss: 1.9111 - val_mean_squared_error: 1.9111
Epoch 2/120
211/211 [==============================] - 40s 188ms/step - loss: 1.3578 - mean_squared_error: 1.3578 - val_loss: 0.8161 - val_mean_squared_error: 0.8161
Epoch 3/120
211/211 [==============================] - 40s 190ms/step - loss: 0.6418 - mean_squared_error: 0.6418 - val_loss: 0.5488 - val_mean_squared_error: 0.5488
Epoch 4/120
211/211 [==============================] - 39s 186ms/step - loss: 0.4368 - mean_squared_error: 0.4368 - val_loss: 0.4168 - val_mean_squared_error: 0.4168
Epoch 5/120
211/211 [==============================] - 39s 183ms/step - loss: 0.3111 - mean_squared_error: 0.3111 - val_loss: 0.3381 - val_mean_squared_error: 0.3381
Epoch 6/120
211/211 [==============================] - 39s 184ms/step - loss: 0.2319 - mean_squared_error: 0.2319 - val_loss: 0.2628 - val_mean_squared_error:

211/211 [==============================] - 60s 283ms/step - loss: 0.0062 - mean_squared_error: 0.0062 - val_loss: 0.0129 - val_mean_squared_error: 0.0129
Epoch 51/120
211/211 [==============================] - 60s 282ms/step - loss: 0.0058 - mean_squared_error: 0.0058 - val_loss: 0.0126 - val_mean_squared_error: 0.0126
Epoch 52/120
211/211 [==============================] - 60s 284ms/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0118 - val_mean_squared_error: 0.0118
Epoch 53/120
211/211 [==============================] - 56s 267ms/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0119 - val_mean_squared_error: 0.0119
Epoch 54/120
211/211 [==============================] - 60s 283ms/step - loss: 0.0046 - mean_squared_error: 0.0046 - val_loss: 0.0112 - val_mean_squared_error: 0.0112
Epoch 55/120
211/211 [==============================] - 60s 282ms/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0115 - val_mean_squared_error: 0.0115
Epoch 56/12

211/211 [==============================] - 43s 203ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0090 - val_mean_squared_error: 0.0090
Epoch 100/120
211/211 [==============================] - 53s 252ms/step - loss: 0.0019 - mean_squared_error: 0.0019 - val_loss: 0.0080 - val_mean_squared_error: 0.0080
Epoch 101/120
211/211 [==============================] - 51s 242ms/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0085 - val_mean_squared_error: 0.0085
Epoch 102/120
211/211 [==============================] - 45s 213ms/step - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0087 - val_mean_squared_error: 0.0087
Epoch 103/120
211/211 [==============================] - 47s 224ms/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0086 - val_mean_squared_error: 0.0086
Epoch 104/120
211/211 [==============================] - 46s 220ms/step - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0080 - val_mean_squared_error: 0.0080
Epoch 

In [13]:
print(model.predict(x_test[10:15]))
print(y_test[10:15])

[[0.60368466 0.5848747 ]
 [0.6053308  0.57265294]
 [0.45513672 0.5834784 ]
 [0.57232857 0.6083819 ]
 [0.6459024  0.6124058 ]]
[[0.6 0.6]
 [0.6 0.6]
 [0.2 0.6]
 [0.6 0.6]
 [0.6 0.6]]


In [9]:
print("Evaluate on test data")
results = model.evaluate(x_test,y_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
18/18 [==============================] - 4s 196ms/step - loss: 0.0075 - mean_squared_error: 0.0075
test loss, test acc: [0.007527012377977371, 0.007527012377977371]


In [8]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('/home/sorozco0612/dev/alan_bot/model_creation/model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp_ung2csg/assets
